In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

In [2]:
from datasets import load_dataset, Dataset

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

Dataset.cleanup_cache_files

<function datasets.arrow_dataset.Dataset.cleanup_cache_files(self) -> int>

In [3]:
dataset_name = "../llama_2"
data_files = {"train": "train.csv", "test": "test.csv"}
dataset = load_dataset(dataset_name, data_files=data_files)

Found cached dataset csv (/home/yehoon/.cache/huggingface/datasets/csv/llama_2-80ca39f4a31b56a4/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['inputs', 'instruction', 'response', 'text'],
        num_rows: 69123
    })
    test: Dataset({
        features: ['inputs', 'instruction', 'response', 'text'],
        num_rows: 62388
    })
})

In [13]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]

train_dataset, test_dataset

(Dataset({
     features: ['inputs', 'instruction', 'response', 'text'],
     num_rows: 69123
 }),
 Dataset({
     features: ['inputs', 'instruction', 'response', 'text'],
     num_rows: 62388
 }))

In [5]:
torch.cuda.is_available()

True

In [6]:
print(f"CUDA is available: {torch.cuda.is_available()}")

model_name = "tiiuae/falcon-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float32,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config=bnb_config,
    trust_remote_code=True)

model.config.use_cache = False
# model.to('cuda')

CUDA is available: True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [8]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

In [9]:
output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [14]:
max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/69123 [00:00<?, ? examples/s]

[codecarbon INFO @ 22:36:07] [setup] RAM Tracking...
[codecarbon INFO @ 22:36:07] [setup] GPU Tracking...
[codecarbon INFO @ 22:36:07] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 22:36:07] [setup] CPU Tracking...
[codecarbon WARNING @ 22:36:07] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 22:36:08] We saw that you have a 12th Gen Intel(R) Core(TM) i9-12900 but we don't know it. Please contact us.
[codecarbon INFO @ 22:36:08] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i9-12900
[codecarbon INFO @ 22:36:08] >>> Tracker's metadata:
[codecarbon INFO @ 22:36:08]   Platform system: Linux-5.19.0-46-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 22:36:08]   Python version: 3.11.4
[codecarbon INFO @ 22:36:08]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 22:36:08]   Available RAM : 125.535 GB
[codecarbon INFO @ 22:36:08]   CPU count: 24
[codecarbon INFO @ 22:36:08]   CPU model: 12th Gen Intel(R) Core(TM) i9-12900
[codecarbo

In [15]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [16]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jangyh0420 (yehoon). Use `wandb login --relogin` to force relogin


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/yehoon/anaconda3/envs/llm/lib/python3.11/site-packages/torch/utils/checkpoint.py:426: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


KeyboardInterrupt: 

In [1]:
import os

In [3]:
output_dir = 'result'

In [4]:
str(os.path.join(output_dir, 'falcon-7b-finetuned-4bit', 'config.json'))

'result/falcon-7b-finetuned-4bit/config.json'